# RNNレイヤの実装

In [1]:
import numpy as np

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')

from common.functions import sigmoid

Google Colab. 上での実行です
「ファイルを選択」から、notebook/commonフォルダのfunctions.pyを選択し、アップロードしてください


Saving functions.py to functions.py


### [演習]
* 以下のRNNレイヤのクラスを完成させましょう

In [2]:
class RNN:
    def __init__(self, Wx, Wh, b):
        """
        Wx : 入力xにかかる重み
        Wh : １時刻前のhにかかる重み
        b : バイアス
        """
        
        # パラメータのリスト
        self.params = [Wx, Wh, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        """
        順伝播計算
        """
        Wx, Wh, b = self.params
        
        # 行列の積　+　行列の積 + バイアス
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        
        # 活性化関数に入れる
        h_next = np.tanh(t)

        # 値の一時保存
        self.cache = (x, h_prev, h_next)
        
        return h_next

    def backward(self, dh_next):
        """
        逆伝播計算
        """
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        # tanhでの逆伝播
        A3 = dh_next * (1 - h_next ** 2) # dh_next * (1 - y^2)
        
        # バイアスbの勾配
        # Nの方向に合計する
        db = np.sum(A3, axis=0)
        
        # 重みWhの勾配
        dWh = np.dot(h_prev.T, A3)
        
        # 1時刻前に渡す勾配
        dh_prev = np.dot(A3, Wh.T)
        
        # 重みWxの勾配
        dWx = np.dot(x.T, A3)
        
        # 入力xに渡す勾配
        dx = np.dot(A3, Wx.T)

        # 勾配をまとめる
        self.grads[0][:] = dWx # 同じメモリ位置に代入
        self.grads[1][:] = dWh # 同じメモリ位置に代入
        self.grads[2][:] = db # 同じメモリ位置に代入

        return dx, dh_prev

In [3]:
D = 1 # 入力層のノード数
H = 5 # 中間層のノード数
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.zeros(H)

# オブジェクトの生成
rnn = RNN(Wx, Wh, b)
print("id of rnn.grads[0]", id(rnn.grads[0]))
print("id of rnn.grads[1]", id(rnn.grads[1]))
print("id of rnn.grads[2]", id(rnn.grads[2]))
print()

# 順伝播計算
N = 4 # バッチサイズ
x = np.random.randn(N, D)
h_prev = np.random.randn(N, H)
h_next = rnn.forward(x, h_prev)
print("h_next=", h_next)
print()

# 逆伝播計算
dh_next = np.random.randn(N, H )
dx, dh_prev = rnn.backward(dh_next)
print("dx=", dx)
print()
print("dh_prev=", dh_prev)
print()

print("id of rnn.grads[0]", id(rnn.grads[0]))
print("id of rnn.grads[1]", id(rnn.grads[1]))
print("id of rnn.grads[2]", id(rnn.grads[2]))
print()


id of rnn.grads[0] 140717520833488
id of rnn.grads[1] 140717520833008
id of rnn.grads[2] 140717520833392

h_next= [[-0.47626258  0.96178137 -0.93342135  0.86533686 -0.99973333]
 [ 0.71498385  0.99928196  0.77574209 -0.96239432 -0.42853182]
 [ 0.78687688  0.9125588   0.88062297  0.68532025  0.97426537]
 [-0.84743239  0.96238737  0.8579326  -0.7565185   0.89792075]]

dx= [[-0.48576441]
 [ 0.4293165 ]
 [ 0.26092907]
 [-0.55222509]]

dh_prev= [[ 0.25273287  0.05030359 -0.25734293 -0.11776721  0.56538343]
 [-0.1104095  -0.56198788 -0.68954851  0.9914727   0.23529424]
 [ 0.11097827  0.02104188 -0.17836857 -0.63604255  0.44622545]
 [ 0.4377263  -0.23758178 -0.31755537  0.53789013  0.05887612]]

id of rnn.grads[0] 140717520833488
id of rnn.grads[1] 140717520833008
id of rnn.grads[2] 140717520833392

